In [3]:
import os
import json
import numpy as np
import skimage.draw
import cv2
import sys

In [18]:
IMAGE_FOLDER = "./images/"
MASK_FOLOER = "./label/"
PATH_ANNOTATION_JSON = './via_project_1357_1395.json'

In [19]:
''' export option one
annotations = json.load(open(PATH_ANNOTATION_JSON, 'r'))
for imgId, item  in annotations.items():
    
    filename = item['filename']
    regions = item['regions']
'''

annotations = json.load(open(PATH_ANNOTATION_JSON, 'r'))
imgs = annotations["_via_img_metadata"]
for imgId in imgs:
    filename = imgs[imgId]['filename']
    regions = imgs[imgId]['regions']
    if len(regions) <= 0:
        continue
    # image path
    image_path = os.path.join(IMAGE_FOLDER, filename)
    # read height and width of the image
    image = cv2.imread(image_path)  # image = skimage.io.imread(image_path)
    height, width = image.shape[:2]

    # create blank mask
    maskImage = np.zeros((height,width), dtype=np.uint8)

    for idx in range(len(regions)):
        polygons = regions[idx]['shape_attributes']
        countOfPoints = len(polygons['all_points_x'])
        points = [None] * countOfPoints
        for i in range(countOfPoints):
            x = int(polygons['all_points_x'][i])
            y = int(polygons['all_points_y'][i])
            points[i] = (x, y)

        contours = np.array(points)
        # iter all coordinates from the map
        for i in range(width):
            for j in range(height):
                if cv2.pointPolygonTest(contours, (i, j), False) > 0:
                    maskImage[j,i] = 255

    savePath = MASK_FOLOER + filename
    # store image
    cv2.imwrite(savePath, maskImage)